In [1]:
import pandas as pd 
import numpy as np
# 从sklearn 调入所需要的包
# from sklearn import datasets

import os
import json
import gc
# os.system('pip install numba')
from numba import jit
#tqdm
# os.system('pip install tqdm')
from tqdm import tqdm_notebook
from tqdm import tqdm

#Integrated model
# os.system('pip install lightgbm')
import lightgbm as lgb
# os.system('pip install catboost==0.15.2')
import catboost as cbt
# os.system('pip install xgboost')
# import xgboost as xgb

#base import 
import numpy as np
import pandas as pd

# about sklearn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler as std
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import f1_score
#about time
import time
import datetime 
from datetime import datetime, timedelta

#Garbage collection
import gc
# scipy
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
import scipy.spatial.distance as dist
#other
from collections import Counter 
from statistics import mode 
    #warning
import warnings
warnings.filterwarnings("ignore")
import json 
import math
from itertools import product
import ast
from sklearn.model_selection import train_test_split #数据分隔出训练集和验证集
import lightgbm as lgb
import numpy as np 
import pandas as pd
#导入精度和召回
from sklearn.metrics import precision_score, recall_score
import lightgbm as lgb
pd.set_option('display.max_columns',None)


In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
# os.system('pip install lightgbm')
import lightgbm as lgb
# os.system('pip install catboost==0.15.2')
import catboost as cbt
os.system('pip install xgboost')
from xgboost import XGBClassifier

xgb_model  = XGBClassifier(learning_rate=0.1,
                      n_estimators=1000,         # 树的个数--1000棵树建立xgboost
                       max_depth=6,               # 树的深度
                       min_child_weight = 1,      # 叶子节点最小权重
                      gamma=0.,                  # 惩罚项中叶子结点个数前的参数
                      subsample=0.8,             # 随机选择80%样本建立决策树
                       colsample_btree=0.8,       # 随机选择80%特征建立决策树
                       objective='binary:logistic', # 指定损失函数
                      scale_pos_weight=1,        # 解决样本个数不平衡的问题
                     random_state=27,# 随机数
                    tree_method='gpu_hist'     
                      )
lgb_model = lgb.LGBMClassifier(
        boosting_type="gbdt", num_leaves=15, reg_alpha=0, reg_lambda=0.,
    max_depth=-1, n_estimators=1500, objective='multiclass',metric= 'multi_error',
    subsample=0.95, colsample_bytree=0.95, subsample_freq=1,
    learning_rate=0.1, random_state=2017
    )
# cbt_model = cbt.CatBoostClassifier(iterations=5000,learning_rate=0.2,max_depth=7,
#                                            l2_leaf_reg=1,verbose=100,early_stopping_rounds=500,
#                                             task_type='GPU',eval_metric='AUC',cat_features=cat_list)



lgb_model_binary = lgb.LGBMClassifier(
        boosting_type="gbdt", num_leaves=150, reg_alpha=0, reg_lambda=0.,
    max_depth=-1, n_estimators=15000, objective='binary',metric= 'error',
    subsample=0.95, colsample_bytree=0.95, subsample_freq=1,
    learning_rate=0.05, random_state=2017
    )

In [4]:
def get_predict_w(model, data, label='label', feature=[], cate_feature=[], random_state=2018, n_splits=5,
                  model_type='lgb'):
#     if 'sample_weight' not in data.keys():
#         data['sample_weight'] = 1
    model.random_state = random_state
    predict_label = ['predict_' + label+str(i) for i in range(0,10)]
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    for i in predict_label:
        data[i] = 0
    test_index = (data[label].isnull()) | (data[label] == -1)
    train_data = data[~test_index].reset_index(drop=True)
    test_data = data[test_index]
    data['agepre']=0
    
    feature_importances = pd.DataFrame()
    feature_importances['feature'] = feature
    
    
    fold=0
    for train_idx, val_idx in kfold.split(train_data):
#         model.random_state = model.random_state + 1
        train_x = train_data.loc[train_idx][feature]
        train_y = train_data.loc[train_idx][label]
        test_x = train_data.loc[val_idx][feature]
        test_y = train_data.loc[val_idx][label]
        if model_type == 'lgb':
            try:
                model.fit(train_x, train_y, eval_set=[(train_x,train_y),(test_x, test_y)], early_stopping_rounds=100,
                          eval_metric='multi_error',
                          categorical_feature=cate_feature,
#                           sample_weight=train_data.loc[train_idx]['sample_weight'],
                          verbose=5
                          )
            except:
                model.fit(train_x, train_y, eval_set=[(test_x, test_y)], early_stopping_rounds=100,
                          eval_metric='multi_error',
                          # categorical_feature=cate_feature,
#                           sample_weight=train_data.loc[train_idx]['sample_weight'],
                          verbose=5)
        elif model_type == 'ctb':
            model.fit(train_x, train_y, eval_set=[(test_x, test_y)], early_stopping_rounds=100,
#                       eval_metric='mae',
#                       cate_features=cate_feature,
                      sample_weight=train_data.loc[train_idx]['sample_weight'],
                      verbose=100)
        elif model_type == 'xgb':
            model.fit(train_x, train_y, eval_set=[(test_x, test_y)], verbose=True, eval_metric='auc',
                      sample_weight=train_data.loc[train_idx]['sample_weight'], early_stopping_rounds=50)
        train_data.loc[val_idx, predict_label] = model.predict_proba(test_x)
        train_data.loc[val_idx, 'agepre'] = model.predict(test_x)
        
        feature_importances['fold_{}'.format(fold + 1)] = model.feature_importances_
        
        fold+=1
        if len(test_data) != 0:
            test_data[predict_label] = test_data[predict_label] + model.predict_proba(test_data[feature])#[:,1]
    test_data[predict_label] = test_data[predict_label] / n_splits
    print(test_data[predict_label].head())
    return train_data[['agepre','age']],train_data[predict_label],test_data[predict_label],feature_importances

In [8]:
path = 'C:/Users/Yan3/Desktop/disaster_recognition/input/'
train_log = pd.read_csv(path+'train.csv',encoding='gbk')
train_log=train_log.drop_duplicates()

In [9]:
train_log.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [10]:
test_log = pd.read_csv(path+'test.csv')

In [11]:
df_log = pd.concat([train_log, test_log])

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

#### 构建Tfidf 特征

In [13]:
X = list(df_log.text.values)
tfv = TfidfVectorizer(sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    norm='l2',
    min_df=0,
    smooth_idf=False,
    max_features=15000)
tfv.fit(X)

TfidfVectorizer(max_features=15000, min_df=0, smooth_idf=False,
                stop_words='english', strip_accents='unicode',
                sublinear_tf=True, token_pattern='\\w{1,}')

In [14]:
X

['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
 'Forest fire near La Ronge Sask. Canada',
 "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
 '13,000 people receive #wildfires evacuation orders in California ',
 'Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school ',
 '#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires',
 '#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas',
 "I'm on top of the hill and I can see a fire in the woods...",
 "There's an emergency evacuation happening now in the building across the street",
 "I'm afraid that the tornado is coming to our area...",
 'Three people died from the heat wave so far',
 'Haha South Tampa is getting flooded hah- WAIT A SECOND I LIVE IN SOUTH TAMPA WHAT AM I GONNA DO WHAT AM I GONNA 

In [15]:
from scipy import sparse
traintext_tfidf = tfv.transform(train_log['text'].values)
sparse.save_npz('./tf_idf_feats/traintext_tfidf.npz',traintext_tfidf)
testtext_tfidf = tfv.transform(test_log['text'].values)
sparse.save_npz('./tf_idf_feats/testtext_tfidf.npz',testtext_tfidf)

In [16]:
train_x = pd.DataFrame()
test_x = pd.DataFrame()
train_data = sparse.hstack((train_x, traintext_tfidf), 'csr')
test_data = sparse.hstack((test_x, testtext_tfidf), 'csr')

In [17]:
train_data[0]

<1x15000 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [18]:
lgb_model_binary = lgb.LGBMClassifier(
        boosting_type="gbdt", num_leaves=150, reg_alpha=0, reg_lambda=0.,
    max_depth=-1, n_estimators=500, objective='binary',metric= 'error',
    subsample=0.95, colsample_bytree=0.95, subsample_freq=1,
    learning_rate=0.025, random_state=2017
    )

In [19]:
y=train_log['target']

In [20]:
fold=0
n_splits=5
test_log['targetpre']=0
train_log['targetpre']=0
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
def run_model():
    for train_idx, val_idx in kfold.split(train_data):
        train_x = train_data[train_idx]
        train_y = y.loc[train_idx]
        test_xt = train_data[val_idx]
        test_yt = y.loc[val_idx]
        lgb_model_binary.fit(train_x, train_y, eval_set=[(train_x,train_y),(test_xt, test_yt)], early_stopping_rounds=100,
                          eval_metric='error',
                          # categorical_feature=cate_feature,
    #                           sample_weight=train_data.loc[train_idx]['sample_weight'],
                          verbose=5)
        train_log.loc[val_idx, 'targetpre'] = lgb_model_binary.predict_proba(test_xt)[:,1]
        test_log['targetpre']+= lgb_model_binary.predict_proba(test_data)[:,1]/n_splits


In [21]:
test_log.head()

,id,keyword,location,text,targetpre
0,0,NaN,NaN,Just happened a terrible car crash,0
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",0
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",0
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,0
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,0


In [22]:
test_log.to_csv('baseline_predict.csv',index=False)

In [23]:
run_model()

Training until validation scores don't improve for 100 rounds
[5]	training's binary_error: 0.430542	valid_1's binary_error: 0.426133
[10]	training's binary_error: 0.300821	valid_1's binary_error: 0.327643
[15]	training's binary_error: 0.259113	valid_1's binary_error: 0.307945
[20]	training's binary_error: 0.243842	valid_1's binary_error: 0.298096
[25]	training's binary_error: 0.234811	valid_1's binary_error: 0.295469
[30]	training's binary_error: 0.2289	valid_1's binary_error: 0.290217
[35]	training's binary_error: 0.222824	valid_1's binary_error: 0.282994
[40]	training's binary_error: 0.216092	valid_1's binary_error: 0.277741
[45]	training's binary_error: 0.213136	valid_1's binary_error: 0.279054
[50]	training's binary_error: 0.208539	valid_1's binary_error: 0.277085
[55]	training's binary_error: 0.207061	valid_1's binary_error: 0.273802
[60]	training's binary_error: 0.203284	valid_1's binary_error: 0.273802
[65]	training's binary_error: 0.200164	valid_1's binary_error: 0.275772
[70]	

[210]	training's binary_error: 0.100985	valid_1's binary_error: 0.233093
[215]	training's binary_error: 0.0985222	valid_1's binary_error: 0.232436
[220]	training's binary_error: 0.0972085	valid_1's binary_error: 0.230466
[225]	training's binary_error: 0.0952381	valid_1's binary_error: 0.231123
[230]	training's binary_error: 0.0932677	valid_1's binary_error: 0.228496
[235]	training's binary_error: 0.0926108	valid_1's binary_error: 0.228496
[240]	training's binary_error: 0.0924466	valid_1's binary_error: 0.22981
[245]	training's binary_error: 0.0898194	valid_1's binary_error: 0.228496
[250]	training's binary_error: 0.08867	valid_1's binary_error: 0.228496
[255]	training's binary_error: 0.0876847	valid_1's binary_error: 0.227183
[260]	training's binary_error: 0.0858785	valid_1's binary_error: 0.225213
[265]	training's binary_error: 0.0842365	valid_1's binary_error: 0.223244
[270]	training's binary_error: 0.0827586	valid_1's binary_error: 0.2239
[275]	training's binary_error: 0.0821018	val

[50]	training's binary_error: 0.212445	valid_1's binary_error: 0.280552
[55]	training's binary_error: 0.208833	valid_1's binary_error: 0.279895
[60]	training's binary_error: 0.205713	valid_1's binary_error: 0.275953
[65]	training's binary_error: 0.20046	valid_1's binary_error: 0.275953
[70]	training's binary_error: 0.191758	valid_1's binary_error: 0.26544
[75]	training's binary_error: 0.187161	valid_1's binary_error: 0.26544
[80]	training's binary_error: 0.178788	valid_1's binary_error: 0.259527
[85]	training's binary_error: 0.170087	valid_1's binary_error: 0.257556
[90]	training's binary_error: 0.162371	valid_1's binary_error: 0.254928
[95]	training's binary_error: 0.156296	valid_1's binary_error: 0.243758
[100]	training's binary_error: 0.150714	valid_1's binary_error: 0.243758
[105]	training's binary_error: 0.145953	valid_1's binary_error: 0.242444
[110]	training's binary_error: 0.144804	valid_1's binary_error: 0.239816
[115]	training's binary_error: 0.140043	valid_1's binary_error: 

In [24]:
train_log.head()

,id,keyword,location,text,target,targetpre
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.656774
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,0.791311
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,0.464131
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,0.667379
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,0.041212


#### 构建COUNTVEC特征

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(min_df=30)
cv.fit(df_log['text'])

CountVectorizer(min_df=30)

In [26]:
train_ta = cv.transform(train_log['text'])
sparse.save_npz('./countvec_feats/traintext_countvec2.npz',train_ta)
test_ta = cv.transform(test_log['text'])
sparse.save_npz('./countvec_feats/testtext_countvec2.npz',test_ta)

In [27]:
train_x = pd.DataFrame()
test_x = pd.DataFrame()
train_data = sparse.hstack((train_x, traintext_tfidf), 'csr')
test_data = sparse.hstack((test_x, testtext_tfidf), 'csr')

In [28]:
fold=0
n_splits=5
test_log['targetpre2']=0
train_log['targetpre2']=0
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
def run_model2():
    for train_idx, val_idx in kfold.split(train_data):
        train_x = train_data[train_idx]
        train_y = y.loc[train_idx]
        test_xt = train_data[val_idx]
        test_yt = y.loc[val_idx]
        lgb_model_binary.fit(train_x, train_y, eval_set=[(train_x,train_y),(test_xt, test_yt)], early_stopping_rounds=100,
                          eval_metric='error',
                          # categorical_feature=cate_feature,
    #                           sample_weight=train_data.loc[train_idx]['sample_weight'],
                          verbose=5)
        train_log.loc[val_idx, 'targetpre2'] = lgb_model_binary.predict_proba(test_xt)[:,1]
        test_log['targetpre2']+= lgb_model_binary.predict_proba(test_data)[:,1]/n_splits

In [29]:
run_model2()

Training until validation scores don't improve for 100 rounds
[5]	training's binary_error: 0.430542	valid_1's binary_error: 0.426133
[10]	training's binary_error: 0.300821	valid_1's binary_error: 0.327643
[15]	training's binary_error: 0.259113	valid_1's binary_error: 0.307945
[20]	training's binary_error: 0.243842	valid_1's binary_error: 0.298096
[25]	training's binary_error: 0.234811	valid_1's binary_error: 0.295469
[30]	training's binary_error: 0.2289	valid_1's binary_error: 0.290217
[35]	training's binary_error: 0.222824	valid_1's binary_error: 0.282994
[40]	training's binary_error: 0.216092	valid_1's binary_error: 0.277741
[45]	training's binary_error: 0.213136	valid_1's binary_error: 0.279054
[50]	training's binary_error: 0.208539	valid_1's binary_error: 0.277085
[55]	training's binary_error: 0.207061	valid_1's binary_error: 0.273802
[60]	training's binary_error: 0.203284	valid_1's binary_error: 0.273802
[65]	training's binary_error: 0.200164	valid_1's binary_error: 0.275772
[70]	

[215]	training's binary_error: 0.0985222	valid_1's binary_error: 0.232436
[220]	training's binary_error: 0.0972085	valid_1's binary_error: 0.230466
[225]	training's binary_error: 0.0952381	valid_1's binary_error: 0.231123
[230]	training's binary_error: 0.0932677	valid_1's binary_error: 0.228496
[235]	training's binary_error: 0.0926108	valid_1's binary_error: 0.228496
[240]	training's binary_error: 0.0924466	valid_1's binary_error: 0.22981
[245]	training's binary_error: 0.0898194	valid_1's binary_error: 0.228496
[250]	training's binary_error: 0.08867	valid_1's binary_error: 0.228496
[255]	training's binary_error: 0.0876847	valid_1's binary_error: 0.227183
[260]	training's binary_error: 0.0858785	valid_1's binary_error: 0.225213
[265]	training's binary_error: 0.0842365	valid_1's binary_error: 0.223244
[270]	training's binary_error: 0.0827586	valid_1's binary_error: 0.2239
[275]	training's binary_error: 0.0821018	valid_1's binary_error: 0.223244
[280]	training's binary_error: 0.0809524	va

[50]	training's binary_error: 0.212445	valid_1's binary_error: 0.280552
[55]	training's binary_error: 0.208833	valid_1's binary_error: 0.279895
[60]	training's binary_error: 0.205713	valid_1's binary_error: 0.275953
[65]	training's binary_error: 0.20046	valid_1's binary_error: 0.275953
[70]	training's binary_error: 0.191758	valid_1's binary_error: 0.26544
[75]	training's binary_error: 0.187161	valid_1's binary_error: 0.26544
[80]	training's binary_error: 0.178788	valid_1's binary_error: 0.259527
[85]	training's binary_error: 0.170087	valid_1's binary_error: 0.257556
[90]	training's binary_error: 0.162371	valid_1's binary_error: 0.254928
[95]	training's binary_error: 0.156296	valid_1's binary_error: 0.243758
[100]	training's binary_error: 0.150714	valid_1's binary_error: 0.243758
[105]	training's binary_error: 0.145953	valid_1's binary_error: 0.242444
[110]	training's binary_error: 0.144804	valid_1's binary_error: 0.239816
[115]	training's binary_error: 0.140043	valid_1's binary_error: 